<a href="https://colab.research.google.com/github/sosucat/sbl-optimizer-colab-simple/blob/main/sbl_optimizer_light_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💡 使用するためにはコピーして下さい
このノートは **閲覧のみ**の設定です。
ノートを使うためには、まず**コピーの保存**を行ってください:

- 左上から:  **`ファイル → ドライブにコピーを保存`**

- Googleドライブに作成されたコピーは、編集・実行可能です。

- Googleアカウントにサインインする必要があります。


# **``🦙 sbl-optimizer light``**<br>[Swell by Light](https://sites.gatech.edu/futurefeelings/2025/07/03/swell-by-light-tei-25-2/)の模様最適化ソフト簡易版
[![PyPI version](https://badge.fury.io/py/sbl-optimizer.svg)](https://badge.fury.io/py/sbl-optimizer)
[![sbl-optimizer](https://img.shields.io/badge/sbl--optimizer-black?logo=github)](https://github.com/sosucat/sbl-optimizer)

[![Homepage](https://img.shields.io/badge/🔗_ホームベージ-black)](https://sites.gatech.edu/futurefeelings/2025/07/03/swell-by-light-tei-25-2/)
[![Author](https://img.shields.io/badge/著者サイト-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)
[![Research paper](https://img.shields.io/badge/研究論文-black?logo=acm)](https://doi.org/10.1145/3689050.3704420)
[![Fabrication](https://img.shields.io/badge/🔗_作り方-black)](https://sites.gatech.edu/futurefeelings/2025/07/23/%e5%85%89%e3%81%a7%e3%83%87%e3%82%b3%e3%83%9c%e3%82%b3%e3%82%82%e3%82%88%e3%81%86%e3%82%92%e4%bd%9c%e3%82%8d%e3%81%86%ef%bc%81/)
[![Watch fabrication demo on YouTube](https://img.shields.io/badge/作り方-750014?logo=youtube)](https://youtu.be/LomVS_jHxl0?feature=shared)

**sbl-optimizer** は、Swell by Lightで画像に忠実なデコボコ模様を作るため、画像の濃淡を修正するソフトです。

**本ノートは簡易版**なので、プログラミングの知識がなくても、画像をアップロードしてセルを実行するだけで、最適化された画像を作ることができます。
できあがった画像を印刷し、光を当てると、元画像の模様と同じようなデコボコ模様ができあがります。

コードの詳細をもっと理解したい方は、上級者向けのColabノートをご覧ください。

[![Advanced Colab](https://img.shields.io/badge/上級者向けColabノート-black?logo=googlecolab)](https://colab.research.google.com/drive/1GLLGjPD7EhUV6evPHUeh6qHe3aNMV47u?usp=sharing)

---

![A printed pattern's shades change as the optimization progresses, and the resulting temperature distribution gets closer to the intended pattern.](https://sites.gatech.edu/futurefeelings/files/2025/03/opt_step.gif)

画像の模様の濃淡を最適化することで、画像の模様（左）に近い、均一な温度分布（右）になり、画像に近いきれいなデコボコ模様ができます。
この例では、最適化前（iteration: 0）は葉の部分が熱くなりすぎている一方で、茎の部分は加熱不足の状態でした。
最適化が進むにつれて、葉の温度は下がり、茎の温度は上昇していき、結果としてより均一な温度分布が実現されています。

---

## 📑 目次
1. [ライブラリのインストール](#scrollTo=vkd63QF86-g3)  
2. [設定の変更](#scrollTo=jNyYGd1sY4Z_)  
3. [模様画像のアップロード](#scrollTo=6dDTuZ9dRhpV)  
4. [最適化の実行 ＆ 結果の表示](#scrollTo=fl5GQtxtfOKb)  
5. [デコボコ模様・温度分布・最適化パフォーマンスの表示](#scrollTo=xNyyC1Q6VD5m)  
6. [最適化された画像のダウンロード](#scrollTo=FtZUXAMQVb_R)
7. [よくある質問 ＆ 問題の解決方法](#scrollTo=-pFXIm8gfsJX)
8. [ライセンス ＆ 引用方法](#scrollTo=q2DhdMj88Kec)
9. [発展](#scrollTo=af-mVoGqfRIf)

---

## 1. ライブラリ・関数の準備
> 本ノートで使用するライブラリ・関数を準備します。

In [ ]:
# Install the latest version from PyPI
!pip install sbl-optimizer

import json
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import importlib.resources as pkg_resources
from google.colab import files
from IPython.display import IFrame, display, Javascript
import base64

from sbl_optimizer.config import Config
from sbl_optimizer.heat_solver import image_to_heat_pattern, optimize
from sbl_optimizer.utils import compute_dims
from sbl_optimizer.io import save_pattern, save_plots

# Locate the bundled config.json inside the installed package
def get_default_config_path():
    return pkg_resources.as_file(
        pkg_resources.files("sbl_optimizer.assets") / "config.json"
    )

# Visualize the initialized print pattern using matplotlib
def visualize_init_pattern(H_init, max_heat):
  plt.figure(figsize=(4,4))
  plt.imshow(H_init/max_heat, cmap='gray_r', vmin=0.0, vmax=1.0)
  plt.axis('off')
  plt.title("Initial print pattern")
  plt.show()

# Visualize the swell pattern and temperature distribution using matplotlib heatmap
def display_plots(plots):
  fig, (ax_swell, ax_temperature) = plt.subplots(1, 2, figsize=(4 * 2, 4))
  img_plot = Image.open(plots[0])
  ax_swell.imshow(img_plot)
  ax_swell.axis('off')
  ax_swell.set_title('Swell Pattern')
  img_plot = Image.open(plots[1])
  ax_temperature.imshow(img_plot)
  ax_temperature.axis('off')
  ax_temperature.set_title('Temperature Distribution [C°]')
  plt.tight_layout()
  plt.show()

# Ask user if they want to download the PDF locally
def download_pdf(pdf_file):
  download_choice = input('Do you want to download the optimized PDF? (y/n): ').strip().lower()
  if download_choice == 'y':
      files.download(str(pdf_file))

# Plot normalized error progression over iterations
def plot_error_progression(errors, cfg):
  errors_norm = [e * 100 / cfg.resolution for e in errors]
  plt.figure(figsize=(5,3))
  plt.plot(errors_norm)
  plt.xlabel('Iteration')
  plt.ylabel('Outlier Cells [%]')
  plt.title('Optimization Error Progress')
  plt.show()

def upload_image():
  """
  Prompt the user to upload an image and save it to the working directory.
  Returns:
      Path object pointing to the uploaded image file.
  """
  uploaded = files.upload()  # Opens upload dialog
  if not uploaded:
      raise ValueError("No file uploaded.")
  # Get the first uploaded filename
  filename = next(iter(uploaded.keys()))
  print(f"Uploaded file: {filename}")
  return Path(filename)


# Set the maximum height of the output
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

##2. ⚙️ 設定の変更

| パラメータ | 種類 | デフォルト | 説明 |
|-----|------|---------|-------------|
| `swell_temperature` | float | 145 [C°] | 膨張温度。使用する膨張ペーストに記載されている、ペーストが膨らみ始める温度に応じて調整してください。 |
| `light_power`       | float | 100.0 [W] |光出力。使用するライトの出力に応じて調節してください。 |
| `light_diameter`    | float | 0.06 [m] | 紙面に当たる光の直径。紙に光を当てて加熱する際に、光が当たる範囲に応じて調整してください。 |
| `alpha`             | float | 5e-07 [m²/s] | 紙の熱流量。厚紙や極端に薄い紙などを使用する際に調節してください。 |
| `verbose`           | int   | 1 | ログ出力の有無。 0：ログなし。 1: ログあり。 |
| `resolution`        | int   | 120000 | 熱シミュレーションの解像度。解像度を下げれば、最適化速度が上がる。解像度を上げれば、より画像に忠実なデコボコ模様になる。 |

> まずはデフォルト値のまま使ってみて、思うようなデコボコ模様ができなければ、変更するのがよいと思います。使用する材料に応じて、以下のセル内の数値を直接編集してください。



In [ ]:
cfg_contents = {
    "swell_temperature": 140.0,
    "light_power": 120.0,
    "light_diameter": 0.08,
    "alpha": 5e-07,
    "verbose": 1,
    "resolution": 120000
}

# Write to a JSON file in the notebook workspace
config_path = Path("custom_config.json")
with open(config_path, "w") as f:
    json.dump(cfg_contents, f, indent=4)
# Load and instantiate Config from this custom file
cfg = Config.from_file(config_path)

## 3. 📁 模様画像のアップロード
> パソコン上からJPGかPNGフォーマットの模様画像をアップロードしてください。
「ファイルを選択」 をクリックして画像を選ぶと、その画像が下に表示されます。

In [ ]:
img_path = upload_image()
img = Image.open(img_path)
mask, H_init, max_heat = image_to_heat_pattern(img_path, cfg)
visualize_init_pattern(H_init, max_heat)

##4. 🔁 最適化の実行 ＆ 結果の表示
> 熱伝導のせいで期待通りの温度にならないところを減らすため、模様の濃淡を最適化します。シミュレーションは、[ここ](#scrollTo=jNyYGd1sY4Z_)で設定したパラメータに基づいて行われます。ここでの「最適化」とは、光による加熱で浮き出るデコボコ模様が、元の模様と一致するようにすることを意味します。最適化後、プレビューが表示されます。

In [ ]:
%%time
# Set the maximum height of the output
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 500})'))

# Compute physical dimensions (for logging)
img_w, img_h, phys_w, phys_h = compute_dims(Image.open(img_path))

# Optimize
T_best, H_best, errors = optimize(mask, H_init, max_heat, img_w, img_h, phys_w, phys_h, cfg)
print(f"Optimization completed in {len(errors)} iterations.")

# Save pattern PDF
pattern_pdf = save_pattern(img_path, H_best, max_heat)
print(f"Saved optimized pattern to: {pattern_pdf}")

# Display the generated PDF inline in Colab
with open(pattern_pdf, "rb") as pdf_file:
    encoded_pdf = base64.b64encode(pdf_file.read()).decode('utf-8')
IFrame(f"data:application/pdf;base64,{encoded_pdf}", width=400, height=400)

##5. 👀 デコボコ模様・温度分布・最適化パフォーマンスの表示

> 予想されるデコボコ模様と温度分布を確認できます。また、温度分布がどのように改善されたかも確認できます。

In [ ]:
# Save and display plots with matplotlib
plots = save_plots(T_best, phys_w, phys_h, img_path, dpi=72)
display_plots(plots)
plot_error_progression(errors, cfg)

##6. 💾 最適化された画像のダウンロード

>結果に満足であれば、最適化された模様画像をダウンロードできます。印刷して、デコボコ模様を作ってみましょう！

In [ ]:
download_pdf(pattern_pdf)

## 7. よくある質問 ＆ 問題の解決方法

Q: どのファイル形式に対応していますか？
A: JPG または PNG 画像をアップロードできます。

Q: 画像がアップロードできません。
A: ファイルを選択 ボタンをクリックして、有効な画像ファイルを選んでください。うまくいかない場合はセルを再実行してください。

Q: 最適化中にエラーが出ました。
A: ファイルが大きすぎないか確認してください。より小さな画像を使い、再実行してみてください。

Q: 最適化にはどのくらい時間がかかりますか？
A: Colab 上では、1～2分ほどかかる場合があります。

ヒント： うまく動作しない場合は、ノートブックを再起動（「ランタイム」→「ランタイムを再起動」）し、最初から再実行してみてください。

以上で問題が解決しない場合、以下からお問い合わせください。

[![@RefreshSource](https://img.shields.io/badge/@RefreshSource-black?logo=x&logoColor=white)](https://x.com/refreshsource)
[![Email](https://img.shields.io/badge/sosuke.ichihashi@gmail.com-black?logo=gmail&logoColor=white)](mailto:sosuke.ichihashi@gmail.com)

## 8. ライセンス ＆ 引用方法

###ライセンス
>本プロジェクトは、[MITライセンス](https://github.com/sosucat/sbl-optimizer-colab-simple/blob/main/LICENSE)のもとで公開されています.


###開発者
>市橋 爽介

> [![市橋 爽介](https://img.shields.io/badge/市橋のサイト-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)
[![@sosucat](https://img.shields.io/badge/@sosucat-black?logo=github&logoColor=white)](https://github.com/sosucat)
[![@RefreshSource](https://img.shields.io/badge/@RefreshSource-black?logo=x&logoColor=white)](https://x.com/refreshsource)


###引用方法
> 研究やプロジェクトで **sbl-optimizer** を使用される場合は、以下を引用してください:
* Sosuke Ichihashi, Noura Howell, and HyunJoo Oh. 2025.\
Swell by Light: An Approachable Technique for Freeform Raised Textures. \
In Proceedings of the Nineteenth International Conference on Tangible, Embedded, and Embodied Interaction (TEI '25). Association for Computing Machinery, New York, NY, USA, Article 45, 1–16. https://doi.org/10.1145/3689050.3704420
```bibtex
@inproceedings{10.1145/3689050.3704420,
author = {Ichihashi, Sosuke and Howell, Noura and Oh, HyunJoo},
title = {Swell by Light: An Approachable Technique for Freeform Raised Textures},
year = {2025},
isbn = {9798400711978},
publisher = {Association for Computing Machinery},
address = {New York, NY, USA},
url = {https://doi.org/10.1145/3689050.3704420},
doi = {10.1145/3689050.3704420},
booktitle = {Proceedings of the Nineteenth International Conference on Tangible, Embedded, and Embodied Interaction},
articleno = {45},
numpages = {16},
keywords = {2.5D fabrication, Personal fabrication, tactile rendering},
location = {Bordeaux / Talence, France},
series = {TEI '25}
}
```

## 9. 発展

**もっと深く知りたい方は、以下をどうぞ！**


🎨 **デコボコ模様の作り方**

> ウェブサイトと動画で、デコボコ模様の作り方を紹介しています。

>[![Fabrication](https://img.shields.io/badge/🔗_作り方-black)](https://sites.gatech.edu/futurefeelings/2025/07/23/%e5%85%89%e3%81%a7%e3%83%87%e3%82%b3%e3%83%9c%e3%82%b3%e3%82%82%e3%82%88%e3%81%86%e3%82%92%e4%bd%9c%e3%82%8d%e3%81%86%ef%bc%81/)
[![Watch fabrication demo on YouTube](https://img.shields.io/badge/作り方-750014?logo=youtube)](https://youtu.be/LomVS_jHxl0?feature=shared)


🔗 **ホームページ**

> Swell by Lightのホームページで、詳細や応用例を紹介しています。

>[![Project page](https://img.shields.io/badge/🔗_ホームページ-black)](https://sites.gatech.edu/futurefeelings/2025/07/03/swell-by-light-tei-25-2/)


📄 **研究論文**

> Swell by Lightの理論や関連研究、応用例については、論文をご参照ください。

>[![Research paper](https://img.shields.io/badge/研究論文-black?logo=acm)](https://doi.org/10.1145/3689050.3704420)



🐐 **上級者向けColabノート**

> 本ノートで使用したコードの詳細を理解したい方は、こちらをご覧ください。

>[![Advanced Colab](https://img.shields.io/badge/上級者向けColabノート-black?logo=googlecolab)](https://colab.research.google.com/drive/1GLLGjPD7EhUV6evPHUeh6qHe3aNMV47u?usp=sharing)



👩‍💻 **ローカル実行用のコマンドラインツール**

> PyPI や GitHub にあるチュートリアルを見て、お使いのパソコン上で sbl-optimizer をより簡単に使ってみましょう。

>[![PyPI version](https://badge.fury.io/py/sbl-optimizer.svg)](https://badge.fury.io/py/sbl-optimizer)
[![GitHub](https://img.shields.io/badge/sbl--optimizer-black?logo=github)](https://github.com/sosucat/sbl-optimizer)


👨‍💻 **ソースコードと問題報告**

> 問題の報告や貢献、最新の開発状況の確認には、ぜひ GitHub リポをご覧ください。

>[![GitHub sbl-optimizer](https://img.shields.io/badge/sbl--optimizer-black?logo=github)](https://github.com/sosucat/sbl-optimizer)
[![GitHub sbl-optimizer-colab-simple](https://img.shields.io/badge/このColabノート-black?logo=github)](https://github.com/sosucat/sbl-optimizer-colab-simple)


🕵️‍♂️ **著者のサイト**

> 市橋 爽介の他の研究もぜひご覧あれ！

>[![著者のサイト](https://img.shields.io/badge/市橋のサイト-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)


*よいモノづくりを！*